<a href="https://colab.research.google.com/github/KChamroeun/Application/blob/main/CalendarRender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

heavenly_stem_characters = {
    "Jia": "甲 ឈើ+",
    "Yi": "乙 ឈើ-",
    "Bing": "丙 ភ្លើង+",
    "Ding": "丁 ភ្លើង-",
    "Wu": "戊 ដី+",
    "Ji": "己 ដី-",
    "Geng": "庚 មាស+",
    "Xin": "辛 មាស-",
    "Ren": "壬 ទឹក+",
    "Gui": "癸 ទឹក-"
      }
heavenly_stem_characters_for_year = {
    0: "庚 Geng មាស+",
    1: "辛 Xin 庚 មាស-",
    2: "壬 Ren ទឹក+",
    3: "癸 Gui ទឹក-",
    4: "甲 Jia ឈើ+",
    5: "乙 Yi ឈើ-",
    6: "丙 Bing ភ្លើង+",
    7: "丁 Ding ភ្លើង-",
    8: "戊 Wu ដី+",
    9: "己 Ji ដី-"
}

print(heavenly_stem_characters_for_year)

print(heavenly_stem_characters)





{'Jia': '甲 ឈើ+', 'Yi': '乙 ឈើ-', 'Bing': '丙 ភ្លើង+', 'Ding': '丁 ភ្លើង-', 'Wu': '戊 ដី+', 'Ji': '己 ដី-', 'Geng': '庚 មាស+', 'Xin': '辛 មាស-', 'Ren': '壬 ទឹក+', 'Gui': '癸 ទឹក-'}


Render Calendar based on Lichun and Winter solstice

In [5]:
!pip install astronomy
!pip install Astronomy

Winter Solstic on 21 or 22nd using skyfield library

| Library    | Accuracy   | Size / Setup   | Best for…                  |
| ---------- | ---------- | -------------- | -------------------------- |
| `astral`   | ±1 min     | tiny, no files | Quick calendar apps        |
| `skyfield` | ±ms        | 6 MB kernel    | Precision, long time spans |
| `pyephem`  | deprecated | —              | Use Skyfield instead       |


“Skyfield” is Python‑only — but you still have good options in Flutter / Dart


| Approach                                                     | How it works                                                                                                                                           | Pros                                                                                    | Cons                                                                                          |
| ------------------------------------------------------------ | ------------------------------------------------------------------------------------------------------------------------------------------------------ | --------------------------------------------------------------------------------------- | --------------------------------------------------------------------------------------------- |
| **1 · Call a small Python micro‑service that runs Skyfield** | Your Flutter app makes HTTPS requests to a FastAPI / Flask / Cloud Function that returns the solstice day (21 or 22 Dec) for a given year.             | • Zero astronomical math to port<br>• Always up‑to‑date if you keep the service current | • Requires hosting a serverless function or VM<br>• Adds network latency / offline dependency |
| **2 · Port the minimal math to Dart**                        | Skyfield ultimately solves *solar ecliptic longitude = 270°.* You can implement the same Meeus algorithms (or use an existing Dart astronomy library). | • Pure Dart (no backend)<br>• Works offline, 100 % client‑side                          | • You must write / port \~150 lines of math or adopt a 3rd‑party lib                          |
| **3 · Use a ready‑made Dart astronomy package**              | `package:astro` and `package:astronomy` exist on pub.dev; both compute solar positions.                                                                | • One‑liner API similar to Skyfield                                                     | • Smaller community than Skyfield; verify accuracy                                            |


In [16]:
!pip install skyfield --quiet

from skyfield.api import load
from skyfield import almanac
from datetime import timezone

ts = load.timescale()
eph = load('de440s.bsp')

def december_solstice(year):
    t0 = ts.utc(year, 12, 19)
    t1 = ts.utc(year, 12, 24)
    times, events = almanac.find_discrete(t0, t1, almanac.seasons(eph))
    for t, e in zip(times, events):
        if e == 3:  # 3 means December solstice
            return t.utc_datetime().replace(tzinfo=timezone.utc)

print(december_solstice(2015))


2015-12-22 04:47:56.661933+00:00


In [ ]:
from datetime import date

def get_solstice_cutoff(year: int) -> tuple:
    """
    Return the correct solstice cutoff for a given year.
    Defaults to Dec 21 unless listed as a Dec 22 solstice year.
    """
    solstice_22_years = {
        2023, 2027, 2031, 2034, 2038, 2042, 2046, 2049,
        2053, 2057, 2061, 2064, 2068, 2072, 2076
    }

    return (12, 22) if year in solstice_22_years else (12, 21)


Lichun calendar

In [ ]:
# -*- coding: utf-8 -*-
"""
bazi_year_by_lichun.py
Flip the Heavenly‑Stem year on LiChun (3 / 4 Feb) instead of Jan 1.
Requires: pip install astronomy      (pure‑Python, tiny)
"""

from datetime import date, timezone, timedelta
from astronomy import Astronomy, Body, Direction

# -------------------------------------------------------------------
# 1.  Astronomical utility: Find the Lìchūn date (local, Asia/Phnom_Penh)

_ASTRO = Astronomy()
_TZ_PP = timezone(timedelta(hours=7))    # UTC+07

def lichun_date(year: int) -> date:
    """
    Return the civil date (Asia/Phnom_Penh) of LiChun for `year`.
    """
    # Search when solar true longitude = 315° — the “Start of Spring”
    # Start from 2 Feb, search forward
    event = _ASTRO.search_hour_angle(
        body=Body.sun,
        longitude=315.0,
        start_date=date(year, 2, 2),
        direction=Direction.next,
    )

    local_dt = event.instant.to_utc_datetime().astimezone(_TZ_PP)
    return date(local_dt.year, local_dt.month, local_dt.day)   # drop time‑of‑day


# -------------------------------------------------------------------
# 2.  Heavenly‑Stem helpers

STEMS = ['甲','乙','丙','丁','戊','己','庚','辛','壬','癸']
REF_YEAR = 1984                 # 1984 = 甲

def heavenly_stem(year: int) -> str:
    """
    Heavenly Stem (single Han character) for a Gregorian YEAR.
    """
    return STEMS[(year - REF_YEAR) % 10]


# -------------------------------------------------------------------
# 3.  Public API: stem for any Gregorian date, LiChun rollover

def heavenly_stem_for_date(greg_date: date) -> str:
    """
    Return Heavenly Stem for the BaZi 'year pillar' containing `greg_date`,
    where the BaZi year starts on LiChun (3 / 4 Feb).
    """
    # Find LiChun for the *Gregorian* year
    lc = lichun_date(greg_date.year)

    # If the date is before LiChun, it still belongs to the previous stem‑year
    bazi_year = greg_date.year if greg_date >= lc else greg_date.year - 1
    return heavenly_stem(bazi_year)


# ---------------------- DEMO ---------------------------------------
if __name__ == "__main__":
    tests = [
        date(2025,  2,  2),
        date(2025,  2,  4),
        date(2026,  2,  3),
        date(2026,  2,  4),
        date(2026,  7,  1),
    ]
    for d in tests:
        print(f"{d}  →  {heavenly_stem_for_date(d)} 年")
